In [1]:
!pip install swig --quiet
!pip install gymnasium --quiet
!pip install stable_baselines3 --quiet
!pip install gymnasium[box2d] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/251b_final_project/jason_colabs/'
# changed last to respective folder

/content/drive/.shortcut-targets-by-id/1RCyQOI1i_gXpHt3VOJnq2dG0-KSqMU10/251b_final_project/jason_colabs


In [5]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

save_dir = '/content/drive/MyDrive/251b_final_project/jason_colabs/'
model_dir = os.path.join(save_dir, 'car_racing_ppo_test1')

In [14]:
import gymnasium as gym
from gymnasium.envs.box2d.car_racing import CarRacing

class CustomCarRacing(CarRacing):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize your custom variable here
        self.on_new_track = 0

    def step(self, action):
        # Reset the indicator at the beginning of the step
        self.on_new_track = 0
        observation, reward, terminated, truncated, info = super().step(action)   # Call the original step

        # Assuming 'reward' incorporates +1000.0/N for every new tile visited and -0.1 otherwise
        # and that no other rewards would precisely equal +1000.0/N in one step,
        # you might infer a new track tile visitation from 'reward' increment.
        # This is a simplification and might not work if the reward calculation is more complex.
        if reward > 0:
            self.on_new_track = 1  # Infer that a new track tile has been visited

        # Check with print statement here
        if self.on_new_track == 1:
            print("On new track tile")
        else:
            print("On grass or old track tile")

        # Add custom tracking info to the 'info' dictionary
        info['on_new_track'] = self.on_new_track

        return observation, reward, terminated, truncated, info

# Adjusting environment setup for vectorized usage
custom_env = CustomCarRacing()  # Create an instance of your custom environment
vec_env = DummyVecEnv([lambda: custom_env])  # Wrap in a dummy vectorized environment

In [15]:
# # Create the environment
# env = gym.make('CarRacing-v2')
# env = DummyVecEnv([lambda: env])  # Wrap in a dummy vectorized environment

# Initialize the agent
model = PPO("CnnPolicy", vec_env, verbose=1)

total_iterations = 10 # change epochs here
time_steps = 10000
best_mean_reward = -float('inf')


Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
#printing the model architecture being used:
print(model.policy)

In [ ]:
# start training

for iteration in range(total_iterations):
    print(f"Training epoch: {iteration + 1}/{total_iterations}") # Changed 'iteration' to 'epoch'
    model.learn(total_timesteps=time_steps, reset_num_timesteps=False)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"Iteration: {iteration + 1}, Mean reward: {mean_reward}, Std reward: {std_reward}")

    if mean_reward > best_mean_reward:
        best_mean_reward = mean_reward
        model.save(model_dir)
        print(f"Model saved at epoch {iteration + 1}") # changed 'iteration' to 'epoch'

In [ ]:
# import os
# save_dir = '/content/drive/MyDrive/CSE_251b/CSE_251b_Final_project/Dima_colabs/'
# model_dir = os.path.join(save_dir, 'car_racing_ppo')
# model.save(model_dir)


## Try a new architecture